# Import modules

In [1]:
from os.path import abspath, join, pardir
from nilearn import datasets
from roi_mean_sub import roi_mean_interface
from bids.layout import BIDSLayout
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.io import DataSink, DataGrabber
from nipype.algorithms.confounds import TSNR
from nipype.interfaces.utility import Function, IdentityInterface
from nilearn.input_data import NiftiLabelsMasker
from nilearn import plotting
from nipype.interfaces import fsl
from nipype.interfaces.utility import Rename

# Define variables for data and atlas

In [2]:
dataset_sub = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
dataset_maps = dataset_sub.maps
labels_sub = dataset_sub.labels

project_path = abspath("data")

layout = BIDSLayout(project_path)

## Show path to output directory

In [3]:
 abspath(join(project_path, pardir,"output"))

'/home/jovyan/work/tSNR_ROI/output'

### Fetch files by subject_id and run_id
        subject_id and run_id are the plain ids as you get them from `BIDSlayout.get_subjects()`
        
    Parameters
    ---------
    layout: BIDSLayout
        layout to use
    subject_id: str
        SubjectID without sub-
    run_id: str
        RunID without run-
        
    Returns
    -------
    preproc,preprocessed_anat, brainmask, subject_id, run_id

### Def function get_files

In [4]:
def get_files(layout, subject_id, run_id):
    brainmask = layout.get(type="brainmask", return_type="file", subject=subject_id, run=run_id)[0]
    preproc = layout.get(type="preproc", return_type="file", subject=subject_id, run=run_id)
    all_preprocs = layout.get(subject=subject_id, type="preproc", return_type="file", modality="anat")
    print(brainmask)
    print(preproc)
    
    
    return preproc, brainmask, subject_id, run_id

### function resample atlas

In [5]:
def resample_atlas(source_img, template_img):
    from nilearn.image import resample_to_img
    resampled_atlas = resample_to_img(source_img, template_img, interpolation='nearest')
    return resampled_atlas

### function save atlas

In [6]:
def save_atlas(resampled_atlas):
    import os
    from os.path import abspath, join, pardir
    import numpy as np
    import nibabel as nib
    from nilearn.image import load_img
    from os.path import abspath
    work_dir = abspath(join("output"))
    atlas = load_img(resampled_atlas)
    save_atlas = atlas.to_filename(abspath('resampled_atlas.nii.gz'))
    saved_atlas = abspath('resampled_atlas.nii.gz')
    return saved_atlas

# Get Data

### Setup Node identitysource

In [7]:
identitysource = Node(IdentityInterface(fields=["subject_id", "run_id"]), name="identitysource")

identitysource.iterables= [('subject_id',layout.get_subjects()),
                           ('run_id', layout.get_runs())]

### Setup Node BIDSDataGrabber

In [8]:
BIDSDataGrabber = Node(Function(function=get_files,  input_names=["layout", "subject_id", "run_id",],
                                       output_names=["preproc", "brainmask", "subject_id", "run_id"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.layout = layout

# Processing Nodes

### setup Node for tsnr map calculation

In [9]:
tsnr = Node(TSNR(regress_poly=2), name='tsnr' )

### setup node for atlas resampling

In [10]:
resample_atlas = Node(name='resample_atlas',
               interface=Function(input_names=['template_img', 'source_img'],
                                  output_names=['resampled_atlas'],
                                  function=resample_atlas))
resample_atlas.inputs.source_img = dataset_maps

### setup node for saving resampled atlas (Nifti1image to nii.gz)

In [11]:
save_atlas = Node(name='save_atlas',
               interface=Function(input_names=['resampled_atlas'],
                                  output_names=['saved_atlas'],
                                  function=save_atlas))

### Setup Node ApplyMask

In [12]:
apply_mask = Node(fsl.ApplyMask(), name="apply_mask")

### Set Atlas for mean calculation

In [13]:
roi_mean = Node(roi_mean_interface, name="roi_mean")

### Setup Node roi_mean
#### calculate mean values using function from roi_mean.py

### rename roi_mean.tsv to sub(subject_id)_run(run_id).tsv 

In [14]:
rename = Node(Rename(format_string="sub%(subject_id)s_run%(run_id)02d.tsv"), name='rename')

### Setup Node datasink

In [15]:
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = abspath(join(project_path, pardir,"output"))

### Connect Nodes, Setup Workflow

In [16]:
tsnr_wf = Workflow(name="TSNR_calculation")
tsnr_wf.base_dir = abspath('work_dir_subcortical')
tsnr_wf.connect([(identitysource, BIDSDataGrabber, [('subject_id','subject_id'),
                                                    ('run_id','run_id')]),  
                 (BIDSDataGrabber, resample_atlas, [('preproc', 'template_img')]),
                 (BIDSDataGrabber, tsnr, [('preproc', 'in_file')]),
                 (BIDSDataGrabber, apply_mask, [('brainmask', 'mask_file')]),
                 (resample_atlas, save_atlas, [('resampled_atlas', 'resampled_atlas')]),
                 (save_atlas, apply_mask, [('saved_atlas', 'in_file')]),
                 (apply_mask, roi_mean, [('out_file', 'roi_file')]),
                 
                 (tsnr, roi_mean, [('tsnr_file', 'data_file')]),
                 (roi_mean, rename, [('out_file', 'in_file')]),
                 (BIDSDataGrabber, rename, [('subject_id','subject_id'),
                                                    ('run_id','run_id')]),
                 (tsnr, datasink, [('tsnr_file', 'subcort.@tsnr_map')]),
                 (save_atlas, datasink, [('saved_atlas', 'subcort.@resampled_atlas')]),
                 (apply_mask, datasink, [('out_file', 'subcort.@masked_atlas')]),
                 (rename, datasink, [('out_file', 'subcort.@mean_roi')])]
         )
         
tsnr_wf.write_graph(graph2use='colored', format='svg', simple_form=True)

181020-18:20:52,183 nipype.workflow INFO:
	 Generated workflow graph: /home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/graph.svg (graph2use=colored, simple_form=True).


'/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/graph.svg'

### Run the workflow

In [ ]:
tsnr_wf.run()

181020-18:20:53,751 nipype.workflow INFO:
	 Workflow TSNR_calculation settings: ['check', 'execution', 'logging', 'monitoring']
181020-18:24:56,371 nipype.workflow INFO:
	 Running serially.
181020-18:24:56,400 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.BIDSDataGrabber" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_11_subject_id_20/BIDSDataGrabber".
181020-18:25:00,174 nipype.workflow INFO:
	 [Node] Cached "TSNR_calculation.BIDSDataGrabber" - collecting precomputed outputs
181020-18:25:00,177 nipype.workflow INFO:
	 [Node] "TSNR_calculation.BIDSDataGrabber" found cached.
181020-18:25:00,179 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.tsnr" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_11_subject_id_20/tsnr".
181020-18:25:00,371 nipype.workflow INFO:
	 [Node] Cached "TSNR_calculation.tsnr" - collecting precomputed outputs
181020-18:25:00,373 nipype.workflow INFO:
	 [Node] "TSNR_calculation.tsnr

Calculationg Means
/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_10_subject_id_20/tsnr/tsnr.nii.gz
/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_10_subject_id_20/apply_mask/resampled_atlas_masked.nii.gz
[[3, 31.272767782651066, 11.500148035650591], [14, 34.157293441544489, 11.070297890015288], [9, 29.20528945287667, 10.663546012340179], [19, 32.172495347664686, 9.9299866804789492], [8, 22.666527120524258, 8.4687220410554112], [18, 16.579919465295216, 7.1915160557157343], [5, 32.654509092009867, 7.8344377868970589], [16, 35.427356406249324, 7.9516999999673503], [4, 26.617877216577948, 7.6398909167950331], [15, 27.068630966131387, 7.0864346727128531]]
181020-18:25:19,542 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.roi_mean".
181020-18:25:19,543 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.rename" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_10_subject_id_20/rename".
181020-1

181020-18:25:36,822 nipype.workflow INFO:
	 [Node] Running "save_atlas" ("nipype.interfaces.utility.wrappers.Function")
181020-18:25:37,234 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.save_atlas".
181020-18:25:37,236 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.apply_mask" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_8_subject_id_20/apply_mask".
181020-18:25:37,440 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.apply_mask".
181020-18:25:37,445 nipype.workflow INFO:
	 [Node] Running "apply_mask" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_8_subject_id_20/save_atlas/resampled_atlas.nii.gz -mas /home/jovyan/work/tSNR_ROI/data/derivatives/fmriprep/sub-20/func/sub-20_task-rest_run-08_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz /home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calc

181020-18:25:54,723 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.BIDSDataGrabber" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_6_subject_id_20/BIDSDataGrabber".
181020-18:25:58,438 nipype.workflow INFO:
	 [Node] Cached "TSNR_calculation.BIDSDataGrabber" - collecting precomputed outputs
181020-18:25:58,440 nipype.workflow INFO:
	 [Node] "TSNR_calculation.BIDSDataGrabber" found cached.
181020-18:25:58,442 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.tsnr" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_6_subject_id_20/tsnr".
181020-18:25:58,631 nipype.workflow INFO:
	 [Node] Cached "TSNR_calculation.tsnr" - collecting precomputed outputs
181020-18:25:58,633 nipype.workflow INFO:
	 [Node] "TSNR_calculation.tsnr" found cached.
181020-18:25:58,634 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.resample_atlas" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_6_s

181020-18:26:18,139 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.rename" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_5_subject_id_20/rename".
181020-18:26:18,484 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.rename".
181020-18:26:18,501 nipype.workflow INFO:
	 [Node] Running "rename" ("nipype.interfaces.utility.base.Rename")
181020-18:26:18,508 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.rename".
181020-18:26:18,508 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.datasink" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_5_subject_id_20/datasink".
181020-18:26:18,566 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.datasink".
181020-18:26:18,591 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
181020-18:26:18,729 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.datasink".
181020-18:26:18,73

...

181020-19:38:17,940 nipype.workflow INFO:
	 [Node] Running "tsnr" ("nipype.algorithms.confounds.TSNR")
181020-19:38:23,498 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.tsnr".
181020-19:38:23,499 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.resample_atlas" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_10_subject_id_02/resample_atlas".
181020-19:38:23,672 nipype.workflow INFO:
	 [Node] Running "resample_atlas" ("nipype.interfaces.utility.wrappers.Function")
181020-19:38:25,542 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.resample_atlas".
181020-19:38:25,543 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.save_atlas" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_10_subject_id_02/save_atlas".
181020-19:38:26,348 nipype.workflow INFO:
	 [Node] Running "save_atlas" ("nipype.interfaces.utility.wrappers.Function")
181020-19:38:26,886 nipype.workflow INFO:
	 [Node] Finished "TSNR_c

181020-19:39:17,649 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
181020-19:39:17,657 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.datasink".
181020-19:39:17,659 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.BIDSDataGrabber" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_8_subject_id_02/BIDSDataGrabber".
181020-19:39:36,556 nipype.workflow INFO:
	 [Node] Running "BIDSDataGrabber" ("nipype.interfaces.utility.wrappers.Function")
/home/jovyan/work/tSNR_ROI/data/derivatives/fmriprep/sub-02/func/sub-02_task-rest_run-08_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
['/home/jovyan/work/tSNR_ROI/data/derivatives/fmriprep/sub-02/func/sub-02_task-rest_run-08_bold_space-MNI152NLin2009cAsym_preproc.nii.gz']
181020-19:39:42,537 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.BIDSDataGrabber".
181020-19:39:42,539 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.tsnr" in "/home/jovyan

181020-19:40:41,135 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.rename" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_7_subject_id_02/rename".
181020-19:40:41,486 nipype.workflow INFO:
	 [Node] Running "rename" ("nipype.interfaces.utility.base.Rename")
181020-19:40:41,492 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.rename".
181020-19:40:41,493 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.datasink" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_7_subject_id_02/datasink".
181020-19:40:41,518 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
181020-19:40:41,527 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.datasink".
181020-19:40:41,528 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.BIDSDataGrabber" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_6_subject_id_02/BIDSDataGrabber".
181020-19:41:00,30

181020-19:42:03,613 nipype.workflow INFO:
	 [Node] Running "roi_mean" ("nipype.interfaces.utility.wrappers.Function")
Calculationg Means
/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_5_subject_id_02/tsnr/tsnr.nii.gz
/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_5_subject_id_02/apply_mask/resampled_atlas_masked.nii.gz
[[3, 36.325405315045671, 15.039663174245128], [14, 35.746716040149458, 15.717591541378889], [9, 34.57521587643258, 11.67307914653421], [19, 43.244317319500091, 14.746045975990615], [8, 27.51856892405063, 9.6122739188659914], [18, 28.301100858800567, 8.9518680773312447], [5, 36.736322275348009, 8.5336402136219256], [16, 49.371563214216209, 11.916480041612751], [4, 35.230826486665947, 11.509079136566044], [15, 38.162873057284877, 11.541347972371334]]
181020-19:42:03,719 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.roi_mean".
181020-19:42:03,721 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.renam

181020-19:43:15,569 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.apply_mask" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_3_subject_id_02/apply_mask".
181020-19:43:15,754 nipype.workflow INFO:
	 [Node] Running "apply_mask" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_3_subject_id_02/save_atlas/resampled_atlas.nii.gz -mas /home/jovyan/work/tSNR_ROI/data/derivatives/fmriprep/sub-02/func/sub-02_task-rest_run-03_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz /home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calculation/_run_id_3_subject_id_02/apply_mask/resampled_atlas_masked.nii.gz
181020-19:43:23,170 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.apply_mask".
181020-19:43:23,173 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.roi_mean" in "/home/jovyan/work/tSNR_ROI/work_dir_subcortical/TSNR_calcula